In [8]:
from config import setup
setup()

In [48]:
import csv
import numpy as np
import sys

sys.stdout = open('salut.txt', 'w')

def divide_by_two(row):
    if len(row)<2:
        return []
    else:
        result = []
        for i in np.arange(0,len(row)-1,1):
            couple = [row[i], row[i+1]]
            result.append(couple)
        return result

def createMatrixListFromCsv():
    list_of_matrix=[]
    list_of_prec=[]
    with open('tests.csv', 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        matrix = []
        prec = []
        on_prec = 0
        
        for line in spamreader:
            if ('---' in line):
                list_of_matrix.append(matrix)
                matrix=[]
                on_prec=1
            else:
                if (line==[]):
                    list_of_prec.append(prec)
                    prec=[]
                    on_prec=0
                else:
                    row = []
                    for cell in line:
                        row.append(int (cell))
                    if on_prec==0:
                        matrix.append(row)
                    else:
                        if (len(row)==2):
                            prec.append(row)
                        else:
                            for divided in divide_by_two(row):
                                prec.append(divided)
        if (matrix!=[]):
            list_of_matrix.append(matrix)
        if (prec!=[]):
            list_of_prec.append(prec)
        
        return list_of_matrix, list_of_prec

createMatrixListFromCsv()

([[[1, 2, 3, 4], [2, 1, 6, 4], [3, 2, 1, 2]], [[1, 2, 3], [4, 5, 6]]],
 [[], [[1, 2]]])

In [46]:
from docplex.cp.model import CpoModel
import numpy as np

def create_model(D, P):

    M=len(D)
    T=len(D[0])
    
    # Create the model
    mdl = CpoModel(name='projetMineure')

    # Create one interval variable per job operation
    x = [[mdl.interval_var(size=D[m][t], optional=True, name="Machine_{}-Job_{}".format(m, t)) for t in range(T)] for m in range(M)]


    # Force no overlap for operations executed on a same machine
    for m in range(M):
        mdl.add(mdl.no_overlap(x[m]))
    
    for t in range(T):
        mdl.add((sum([mdl.presence_of(x[m][t]) for m in range(M)])==1))
    
    for p in P:
        for m in range(M):
            [mdl.add(mdl.end_before_start(x[m][p[0]],x[o][p[1]])) for o in range(M)]
        
    # Minimize termination date
    flatten_x = np.array(x).flatten()
    
    mdl.add(mdl.minimize(mdl.max([mdl.end_of(flatten_x[t]) for t in range(len(flatten_x))])))
        
    return mdl, x

#mdl, x = create_model([[1,5,5],[5,1,5],[5,5,1]])
mdl, x = create_model([[1,2,3,4,5,6,7,8,9],[9,8,7,6,5,4,3,2,1],[2,3,9,8,7,5,1,4,2],
                       [7,4,1,8,5,2,9,6,3],[3,6,9,2,5,8,1,4,7],[6,5,4,3,2,1,9,8,7]],
                     [[1,0], [8,7], [3,5], [5,4]])
sol=mdl.solve()
if (sol):
   
    sol.print_solution()
            
else:
    print("error: No solution found")

In [47]:
import sys

sys.stdout = open('outputFileTest.txt', 'w')

list_of_matrix, list_of_prec = createMatrixListFromCsv()
id_matrix=0

if (len(list_of_matrix)!=len(list_of_prec)):
    print("Pb dans la creation des matrices d'entree")
else :
    for i in range(len(list_of_matrix)):
        matrix=list_of_matrix[i]
        prec=list_of_prec[i]
        if matrix!=[]:
            mdl, x = create_model(matrix, prec)
            sol=mdl.solve()
            if (sol):   
                #id_Matrice
                print(id_matrix, end='')
                print(",", end='')

                #Nombre de Machines
                print(len(matrix), end='')
                print(",", end='')

                #Nombre de Tâches
                print(len(matrix[0]))

                #Separation id_Matrice et Matrice 
                print("---")

                var=sol.get_all_var_solutions()
                for v in var:
                    if not(v.is_absent()):
                        name=v.get_name().split("-")

                        #Numero de la machine
                        print(name[0].split("_")[1]+",", end='')

                        #Numero de la tache
                        print(name[1].split("_")[1]+",", end='')

                        #Temps du debut de la tache
                        print(v.get_start(), end='')     
                        print(",", end='')

                        #Temps de la fin de la tache
                        print(v.get_end())

                #Separation Matrice et Résultats
                print("---")

                #Temps d'execution
                print(sol.get_solve_time(), end='')
                print(",", end='')

                #Resultat Optimal
                print(sol.get_objective_values()[0])

                print()


            else:
                print("error: No solution found")  
            id_matrix+=1